In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./CycleSR/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install --upgrade pip
!pip uninstall scikit-image
!pip install scikit-image
import skimage
print(skimage.__version__)
# skimage 0.19.0 is compulsory

Found existing installation: scikit-image 0.19.2
Uninstalling scikit-image-0.19.2:
  Would remove:
    /usr/local/bin/skivi
    /usr/local/lib/python3.7/dist-packages/doc/ext/*
    /usr/local/lib/python3.7/dist-packages/scikit_image-0.19.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_image.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/skimage/*
Proceed (Y/n)? n
0.19.2


In [ ]:
import argparse
import glob
import random
import os
from PIL import Image
import numpy as np
import time
import datetime
import sys
import torch
import math

from math import sqrt
import numpy as np 
import torch.nn.init as init
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable
import itertools
import matplotlib.pyplot as plt
import pdb
import skimage.metrics
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import utils
from utils import set_requires_grad, weights_init_normal, ReplayBuffer, LambdaLR, save_sample, eval, eval_6m, train_eval
from dataset import ImageDataset, ImageDataset_6mm
import ssim


In [ ]:
class MeanShift(nn.Conv2d):
    def __init__(self, rgb_mean, sign):
        super(MeanShift, self).__init__(1, 1, kernel_size=1)
        self.weight.data = torch.eye(1).view(1, 1, 1, 1)
        self.bias.data = float(sign) * torch.Tensor(rgb_mean)

        # Freeze the MeanShift layer
        for params in self.parameters():
            params.requires_grad = False

class _Residual_Block(nn.Module): 
    def __init__(self):
        super(_Residual_Block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x): 
        identity_data = x
        output = self.relu(self.conv1(x))
        output = self.conv2(output)
        output *= 0.1
        output = torch.add(output,identity_data)
        return output 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        rgb_mean = (0.4488)
        # self.sub_mean = MeanShift(rgb_mean, -1)

        self.conv_input = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

        self.residual = self.make_layer(_Residual_Block, 32)

        self.conv_mid = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)

        self.upscale4x = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
            nn.Conv2d(in_channels=256, out_channels=256*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PixelShuffle(2),
        )

        self.conv_output = nn.Conv2d(in_channels=256, out_channels=1, kernel_size=3, stride=1, padding=1, bias=False)

        # self.add_mean = MeanShift(rgb_mean, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                if m.bias is not None:
                    m.bias.data.zero_()

    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        out = x
        out = self.conv_input(out)
        residual = out
        out = self.conv_mid(self.residual(out))
        out = torch.add(out,residual)
        out = self.upscale4x(out)
        out = self.conv_output(out)
        # out += x
        return out

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_test=None, unaligned=False, mode='test'):
        self.transformA = transforms.Compose(transforms_test+[transforms.Resize((64, 64), interpolation=Image.BICUBIC)])
        self.transformB = transforms.Compose(transforms_test)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, '6x6_256/') + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, '3x3_256/') + '/*.*'))

    def __getitem__(self, index):
        path_A = self.files_A[index % len(self.files_A)]
        img_A = Image.open(path_A).convert('L')
        # print(self.files_A[index % len(self.files_A)])

        item_A = self.transformA(img_A)
        path_B = path_A.replace("_3.", "_6.").replace("6x6", "3x3")

        item_B = self.transformB(Image.open(path_B).convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

class ImageDataset_6mm(Dataset):
    def __init__(self, root, transforms_A=None, transforms_B=None, unaligned=False, mode='train'):
        self.transformA = transforms.Compose(transforms_A)
        self.transformB = transforms.Compose(transforms_B)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, 'LR') + '/*.*'))
        # self.files_B = sorted(glob.glob(root + '/*.*'))

    def __getitem__(self, index):
        path_A = self.files_A[index % len(self.files_A)]

        path_B = path_A
        path_B = path_B.replace("_lr.", "_hr.").replace("LR", "HR")

        item_A = self.transformA(Image.open(path_A).convert('L'))
        item_B = self.transformB(Image.open(path_B).convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return len(self.files_A)


test_path = "./dataset/test/"
transforms_test = [ 
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
                ]
dataset = ImageDataset(test_path, transforms_test=transforms_test, unaligned=True)
l = len(dataset)
train, valid = torch.utils.data.random_split(dataset=dataset, lengths=[200, l-200])
dataloader = DataLoader(train, batch_size=1, shuffle=True)


###############
transforms_A = [ 
                transforms.ToTensor(),
                transforms.CenterCrop(256),
                transforms.Resize((64, 64), interpolation=Image.BICUBIC),
                transforms.Normalize((0.5), (0.5))
                ]
transforms_B = [ 
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5)),
                transforms.CenterCrop(256)]

test_path = "./dataset/evalution_6mm/parts"
test_dataset = ImageDataset_6mm(test_path, transforms_A=transforms_A, transforms_B=transforms_B, unaligned=False)

In [ ]:
model = Net()
model.apply(weights_init_normal)
criterion = nn.L1Loss(size_average=False)
model = model.cuda()
criterion = criterion.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0006)

In [ ]:
def eval_6m(model, dataset=test_dataset):
    n = len(dataset)
    num, psnr, ssim, mse, nmi= 0, 0, 0, 0, 0
    model.eval()
    for i in range(n):
        img = dataset[i]['A'].unsqueeze(0).cuda()
        gt = dataset[i]['B'].unsqueeze(0).cuda()


        y = model(img)

        yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
        
        # print(gt.shape)
        # print("fdsafasd fsdaf")
        gtimg = gt.cpu().detach().numpy().squeeze(0).squeeze(0)
        psnr += (skimage.metrics.peak_signal_noise_ratio(yimg, gtimg,data_range=2))
        ssim += (skimage.metrics.structural_similarity(yimg, gtimg))
        mse += (skimage.metrics.mean_squared_error(yimg, gtimg))
        nmi += (skimage.metrics.normalized_mutual_information(yimg, gtimg))
        num += 1
    print(" PSNR: %.4f SSIM: %.4f MSE: %.4f NMI: %.4f"%(psnr/num, ssim/num, mse/num, nmi/num))

def eval_3m(model, dataset=valid):
    n = len(dataset)
    num, psnr, ssim, mse, nmi= 0, 0, 0, 0, 0
    model.eval()
    for i in range(n):
        img = dataset[i]['A'].unsqueeze(0).cuda()
        gt = dataset[i]['B'].unsqueeze(0).cuda()
        y = img

        y = model(img)

        yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
        
        gtimg = gt.cpu().detach().numpy().squeeze(0).squeeze(0)
        psnr += (skimage.metrics.peak_signal_noise_ratio(yimg, gtimg,data_range=2))
        ssim += (skimage.metrics.structural_similarity(yimg, gtimg))
        mse += (skimage.metrics.mean_squared_error(yimg, gtimg))
        nmi += (skimage.metrics.normalized_mutual_information(yimg, gtimg))
        num += 1
    print(" PSNR: %.4f SSIM: %.4f MSE: %.4f NMI: %.4f"%(psnr/num, ssim/num, mse/num, nmi/num))

In [ ]:
Tensor = torch.cuda.FloatTensor
input_A = Tensor(1, 1, 64, 64)
input_B = Tensor(1, 1, 256, 256)

In [31]:
for epoch in range(50):
    real_out, fake_out = None, None
    eval_3m(model)
    eval_6m(model)
    for i, batch in enumerate(dataloader):
        lr = Variable(input_A.copy_(batch['A']))
        target = Variable(input_B.copy_(batch['B']))
        # print(lr.size(), model(lr).size())
        loss = criterion(model(lr), target)
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
    print("------------"+str(epoch+1)+"--------------")
    
        # break


 PSNR: 10.2388 SSIM: 0.0001 MSE: 0.3799 NMI: 1.0093
 PSNR: 10.3458 SSIM: 0.0002 MSE: 0.3714 NMI: 1.0080
------------1--------------
 PSNR: 16.9567 SSIM: 0.3076 MSE: 0.0816 NMI: 1.0477
 PSNR: 16.8462 SSIM: 0.3522 MSE: 0.0841 NMI: 1.0518
------------2--------------
 PSNR: 17.0868 SSIM: 0.3122 MSE: 0.0792 NMI: 1.0499
 PSNR: 16.9133 SSIM: 0.3564 MSE: 0.0828 NMI: 1.0531
------------3--------------
 PSNR: 17.0390 SSIM: 0.3301 MSE: 0.0802 NMI: 1.0509
 PSNR: 16.9584 SSIM: 0.3624 MSE: 0.0819 NMI: 1.0543
------------4--------------
 PSNR: 17.2132 SSIM: 0.3290 MSE: 0.0770 NMI: 1.0512
 PSNR: 17.0697 SSIM: 0.3677 MSE: 0.0799 NMI: 1.0544
------------5--------------
 PSNR: 17.1835 SSIM: 0.3331 MSE: 0.0775 NMI: 1.0504
 PSNR: 16.8946 SSIM: 0.3660 MSE: 0.0831 NMI: 1.0532
------------6--------------
 PSNR: 17.1877 SSIM: 0.3299 MSE: 0.0775 NMI: 1.0524
 PSNR: 17.2370 SSIM: 0.3724 MSE: 0.0769 NMI: 1.0547
------------7--------------
 PSNR: 17.2383 SSIM: 0.3358 MSE: 0.0765 NMI: 1.0515
 PSNR: 16.9496 SSIM: 0.3

KeyboardInterrupt: ignored